# Models

And now - this colab unveils the heart (or the brains?) of the transformers library - the models:

https://colab.research.google.com/drive/1hhR9Z-yiqjUe7pJjVQw4c74z_V3VchLy?usp=sharing

This should run nicely on a low-cost or free T4 box.

my colab notebook:

https://colab.research.google.com/drive/1-dUyawPg7d23opO_mBEwqsKoOEbKU-9j?usp=sharing

## Some notes on the code:

```python
from transformers import BitsAndBytesConfig
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                            # 开启 4-bit 加载
    bnb_4bit_use_double_quant=True,               # 使用双重量化
    bnb_4bit_compute_dtype=torch.bfloat16,        # 推理时使用 bfloat16
    bnb_4bit_quant_type="nf4"                     # 使用 NF4 量化类型
)
```
`BitsAndBytesConfig` 是 `Transformers` 提供的一个类，用于配置和控制通过 `bitsandbytes` 库对模型进行量化加载的方式，常用于节省显存、加速推理。

🔹`load_in_4bit=True`

- 表示将模型加载为 4-bit 量化格式（而不是常见的 FP16、INT8 等），极大地减少显存使用（比如一个 13B 模型能在 24GB GPU 上运行）。

- 需依赖 `bitsandbytes` 库。

🔹 `bnb_4bit_use_double_quant=True`

- 启用 双重量化（Double Quantization）：

- 先将权重量化为更低精度，再对量化器本身也进行压缩。

- 进一步减少模型体积，但会稍微增加计算复杂度。

- 在多 GPU 部署下可显著节省内存。

🔹 `bnb_4bit_compute_dtype=torch.bfloat16`

- 指定模型推理时的计算精度。

- `torch.bfloat16`（Brain Float 16）在新一代 GPU（如 A100、H100、4090）上具有良好支持，保留动态范围的同时节省显存。

- 可替代 torch.float16（标准半精度），但对精度更友好。

🔹 `bnb_4bit_quant_type="nf4"`

- 选择量化方法为 NF4（Normal Float 4）：

- 是 `bitsandbytes` 提供的一种高级 4-bit 量化方法，相比传统 fp4 更准确。

- 具有更好的保真度（对大模型效果影响更小）。

这种设置非常适合：

- 加载大语言模型（如 LLaMA-2、Mistral、Baichuan 等）到显存有限的 GPU（24GB 或更少）。

- 节省内存同时保持较高推理性能。

- 适合推理和轻量级微调（LoRA）任务。

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
  del model, inputs, tokenizer, outputs, streamer
  gc.collect()
  torch.cuda.empty_cache()
  ```

🔹 加载分词器（Tokenizer）
```python
tokenizer = AutoTokenizer.from_pretrained(model) # 根据模型名自动加载相应的分词器。
tokenizer.pad_token = tokenizer.eos_token
```
`tokenizer.pad_token = tokenizer.eos_token`：将 \<pad\> token 设置为 \<eos\>（结束符）。一些模型没有显式的 pad token，这样可以避免错误。

🔹 应用聊天模板并转为张量
```python
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True
).to("cuda")
```
`apply_chat_template`：将对话 messages 转换为模型所需的格式（如 ChatML、Llama 2 的 prompt 格式等）。

`return_tensors="pt"`：将处理后的文本转换为 PyTorch 张量。

`add_generation_prompt=True`：为生成添加一个“提示位”（让模型知道要继续生成回复）。

`.to("cuda")`：将输入数据移动到 GPU。

🔹 创建流式输出器
```python
streamer = TextStreamer(tokenizer)
```
`TextStreamer` 是一个用于实时输出生成文本的工具，它会在模型生成的过程中实时打印结果。
（通常是 Hugging Face 的 `transformers.TextStreamer`。）

🔹 加载模型
```python
model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map="auto",
    quantization_config=quant_config
)
```
`AutoModelForCausalLM.from_pretrained(...)`：加载一个自回归语言模型（Causal LM）。

`device_map="auto"`：自动将模型分布到可用的 GPU 上。

`quantization_config=quant_config`：使用量化配置加载模型（例如 4-bit、8-bit 以节省显存）。

🔹 文本生成
```python
outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
```
`model.generate(...)`：生成最多 80 个新 token 的回复。

`streamer=streamer`：边生成边打印。

🔹 清理内存
```python
del model, inputs, tokenizer, outputs, streamer
gc.collect()
torch.cuda.empty_cache()
```
删除变量，释放显存。

`gc.collect()`：触发 Python 的垃圾回收机制。

`torch.cuda.empty_cache()`：清理 CUDA 缓存，防止显存占用持续上升。

## 解释LlamaForCausalLM的结构

这段模型结构是一个 **LLaMA 大语言模型（LLM）用于自回归语言建模（Causal LM）** 的精简表示，模型为 4-bit 量化版本。我们逐层解释这份架构：

---

### 🔧 顶层模型

```python
LlamaForCausalLM(
  (model): LlamaModel(...)
  (lm_head): Linear(...)
)
```

这是 Hugging Face `transformers` 中的标准封装：

* `LlamaForCausalLM`：用于语言建模（如生成、补全），继承自 `PreTrainedModel`。
* `model`: 实际的 Transformer 主体（`LlamaModel`）。
* `lm_head`: 语言建模头部（将隐藏层映射到词表的线性层），用于预测下一个 token。

---

### 🧱 嵌入层

```python
(embed_tokens): Embedding(128256, 4096)
```

* `Embedding(vocab_size=128256, hidden_size=4096)`：

  * 将输入 token ID 映射为维度为 4096 的向量。
  * `128256` 是词表大小。
  * `4096` 是模型的隐藏维度。

---

### 🔄 Transformer 编码器层

```python
(layers): ModuleList(
  (0-31): 32 x LlamaDecoderLayer(...)
)
```

* 模型有 32 层（表示是 **LLaMA-13B** 模型）。
* 每一层是一个 `LlamaDecoderLayer`，类似于标准的 Transformer 解码器层，但为 **自回归任务** 特别定制。

---

### 🧠 单层结构：LlamaDecoderLayer

以每一层为例，结构如下：

#### 🔸 注意力机制（Self-Attention）

```python
(self_attn): LlamaAttention(
  (q_proj): Linear4bit(4096 → 4096)
  (k_proj): Linear4bit(4096 → 1024)
  (v_proj): Linear4bit(4096 → 1024)
  (o_proj): Linear4bit(4096 → 4096)
)
```

* 标准多头注意力机制（Multi-head Self-Attention）。
* Q（查询）、K（键）、V（值）分别是线性投影。
* 注意的是：**这里用了 4-bit 量化（Linear4bit）**，大幅减少显存和内存占用。
* `k_proj` 和 `v_proj` 的输出维度是 1024，意味着可能用了 4 个头（4096 ÷ 1024 = 4）。

#### 🔸 前馈网络（MLP）

```python
(mlp): LlamaMLP(
  (gate_proj): Linear4bit(4096 → 14336)
  (up_proj): Linear4bit(4096 → 14336)
  (down_proj): Linear4bit(14336 → 4096)
  (act_fn): SiLU()
)
```

* 前馈层结构采用 Gated Feedforward，类似于：

  $$
  \text{FFN}(x) = \text{SiLU}(x W_1) \cdot (x W_2)
  $$
* `14336` 是中间维度（扩展层），非常宽，典型于 LLaMA 模型。
* `SiLU` 是激活函数，也称为 `Swish`，适合大模型训练。

#### 🔸 层归一化（LayerNorm）

```python
(input_layernorm): LlamaRMSNorm((4096,), eps=1e-5)
(post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-5)
```

* 使用了 **RMSNorm（均方根归一化）** 而不是传统的 LayerNorm。
* RMSNorm 更轻量，不依赖均值，只使用 L2 范数进行归一化，适合大模型加速。

---

### 🔁 最终归一化和位置编码

```python
(norm): LlamaRMSNorm((4096,), eps=1e-5)
(rotary_emb): LlamaRotaryEmbedding()
```

* 整个 Transformer 的最后一个层归一化。
* `rotary_emb` 表示使用 **旋转位置编码（RoPE）**，替代绝对位置编码，适合支持无限长输入和高效推理。

---

### 🎯 输出层（语言建模头）

```python
(lm_head): Linear(4096 → 128256)
```

* 将 Transformer 的输出映射回词表维度（预测下一个 token 的概率分布）。
* 没有 bias（偏置项），进一步减少参数。

---

### 🔢 量化层说明：Linear4bit

* 所有的线性层都是 `Linear4bit`，来自 [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)。
* 表示这部分参数是用 4-bit 精度存储和推理的，而不是 float16 或 float32。
* 极大减少显存使用（通常是 3\~4 倍），使得在消费级 GPU（如 RTX 3090, 4090）上运行大模型成为可能。

---

### ✅ 总结（核心信息）

| 项目       | 内容                      |
| -------- | ----------------------- |
| 模型类型     | LLaMA 自回归语言模型           |
| 参数精度     | 4-bit 量化（bitsandbytes）  |
| 层数       | 32 层                    |
| 隐藏层维度    | 4096                    |
| 词表大小     | 128256                  |
| MLP 扩展维度 | 14336                   |
| 激活函数     | SiLU（Swish）             |
| 归一化方法    | RMSNorm                 |
| 位置编码     | Rotary Embedding (RoPE) |

